# Лабораторная работа 6

## Практикум 6. Задание

В данной лабораторной работе будут выполнены следующие задачи:

### Задание 1: Загрузка и предобработка данных
- Загрузить датасет с реальными данными
- Провести предобработку и очистку данных
- Выполнить разведочный анализ данных (EDA)

### Задание 2: Анализ временных рядов
- Построить временные ряды для ключевых показателей
- Выполнить декомпозицию временных рядов
- Провести анализ трендов и сезонности

### Задание 3: Кластерный анализ
- Применить различные алгоритмы кластеризации (K-means, DBSCAN, иерархическая)
- Определить оптимальное количество кластеров
- Визуализировать результаты кластеризации

### Задание 4: Анализ аномалий
- Выявить аномальные значения в данных
- Применить методы обнаружения выбросов
- Проанализировать причины аномалий

### Задание 5: Регрессионный анализ
- Построить модели регрессии для предсказания целевых переменных
- Оценить качество моделей
- Проанализировать важность признаков

### Задание 6: Классификация
- Обучить модели классификации
- Сравнить различные алгоритмы
### Задание 7: Ансамбли методов
- Создать ансамбль моделей
- Оценить эффективность ансамбля

### Задание 8: Визуализация и интерпретация результатов
- Создать интерактивные визуализации
- Подготовить итоговый отчет с выводами

In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Машинное обучение
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, VotingClassifier
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    mean_squared_error, r2_score, silhouette_score, adjusted_rand_score
)

# Анализ временных рядов
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Анализ аномалий
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor

# Настройка отображения
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("Все библиотеки успешно импортированы!")

## Задание 1: Загрузка и предобработка данных

В этом разделе загружаем данные и проводим их предварительную обработку.

In [ ]:
# Задание 1: Загрузка и предобработка данных

# Загрузка данных (замените на путь к вашему файлу)
# df = pd.read_csv('path_to_your_data.csv')
# df = pd.read_excel('path_to_your_data.xlsx')

# Для демонстрации создадим реалистичный датасет с временными рядами
np.random.seed(42)
n_samples = 1000
dates = pd.date_range('2020-01-01', periods=n_samples, freq='D')

# Создаем данные с трендом и сезонностью
trend = np.linspace(100, 200, n_samples)
seasonal = 10 * np.sin(2 * np.pi * np.arange(n_samples) / 365.25)
noise = np.random.normal(0, 5, n_samples)
target = trend + seasonal + noise

# Создаем дополнительные признаки
data = {
    'date': dates,
    'target': target,
    'feature1': np.random.normal(50, 15, n_samples),
    'feature2': np.random.normal(30, 10, n_samples),
    'feature3': np.random.normal(20, 8, n_samples),
    'category': np.random.choice(['A', 'B', 'C'], n_samples, p=[0.5, 0.3, 0.2]),
    'region': np.random.choice(['North', 'South', 'East', 'West'], n_samples)
}

df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

print(f"Форма датафрейма: {df.shape}")
print(f"Колонки: {list(df.columns)}")
print(f"Период данных: {df.index.min()} - {df.index.max()}")
print("\nПервые 5 строк:")
df.head()

## Задание 2: Исследовательский анализ данных

Проводим анализ структуры данных, проверяем на пропуски и выбросы.

In [ ]:
# Задание 2: Анализ временных рядов

# 2.1 Построение временных рядов
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# График основного временного ряда
axes[0, 0].plot(df.index, df['target'], linewidth=1, alpha=0.8)
axes[0, 0].set_title('Временной ряд целевой переменной')
axes[0, 0].set_xlabel('Дата')
axes[0, 0].set_ylabel('Значение')
axes[0, 0].grid(True, alpha=0.3)

# Графики признаков
for i, col in enumerate(['feature1', 'feature2', 'feature3']):
    row, col_idx = (0, 1) if i == 0 else (1, i-1)
    axes[row, col_idx].plot(df.index, df[col], linewidth=1, alpha=0.8)
    axes[row, col_idx].set_title(f'Временной ряд {col}')
    axes[row, col_idx].set_xlabel('Дата')
    axes[row, col_idx].set_ylabel('Значение')
    axes[row, col_idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 2.2 Декомпозиция временного ряда
decomposition = seasonal_decompose(df['target'], model='additive', period=365)

fig, axes = plt.subplots(4, 1, figsize=(15, 12))
decomposition.observed.plot(ax=axes[0], title='Исходный ряд')
decomposition.trend.plot(ax=axes[1], title='Тренд')
decomposition.seasonal.plot(ax=axes[2], title='Сезонность')
decomposition.resid.plot(ax=axes[3], title='Остатки')

for ax in axes:
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 2.3 Статистические тесты
print("Статистический анализ временного ряда:")
print(f"Среднее значение: {df['target'].mean():.2f}")
print(f"Стандартное отклонение: {df['target'].std():.2f}")
print(f"Коэффициент вариации: {df['target'].std()/df['target'].mean()*100:.2f}%")

# Тест на стационарность
adf_result = adfuller(df['target'])
print(f"\nТест Дики-Фуллера:")
print(f"ADF статистика: {adf_result[0]:.4f}")
print(f"p-value: {adf_result[1]:.4f}")
print(f"Критические значения: {adf_result[4]}")
print(f"Ряд {'стационарен' if adf_result[1] < 0.05 else 'нестационарен'} (p < 0.05)")

## Задание 3: Визуализация данных

Создаем различные типы визуализаций для анализа данных.

In [ ]:
# Задание 3: Кластерный анализ

# 3.1 Подготовка данных для кластеризации
X_cluster = df[['feature1', 'feature2', 'feature3']].values
scaler_cluster = StandardScaler()
X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)

print(f"Размер данных для кластеризации: {X_cluster_scaled.shape}")

# 3.2 Определение оптимального количества кластеров (метод локтя)
inertias = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_cluster_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_cluster_scaled, kmeans.labels_))

# График метода локтя и силуэтного анализа
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(K_range, inertias, 'bo-')
axes[0].set_title('Метод локтя')
axes[0].set_xlabel('Количество кластеров')
axes[0].set_ylabel('Инерция')
axes[0].grid(True, alpha=0.3)

axes[1].plot(K_range, silhouette_scores, 'ro-')
axes[1].set_title('Силуэтный анализ')
axes[1].set_xlabel('Количество кластеров')
axes[1].set_ylabel('Силуэтный коэффициент')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Выбираем оптимальное количество кластеров
optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"Оптимальное количество кластеров: {optimal_k}")
print(f"Лучший силуэтный коэффициент: {max(silhouette_scores):.3f}")

## Задание 4: Построение моделей машинного обучения

Создаем и обучаем различные модели для решения задачи.

In [ ]:
# 3.3 Применение различных алгоритмов кластеризации
algorithms = {
    'K-Means': KMeans(n_clusters=optimal_k, random_state=42, n_init=10),
    'DBSCAN': DBSCAN(eps=0.5, min_samples=5),
    'Agglomerative': AgglomerativeClustering(n_clusters=optimal_k)
}

clustering_results = {}

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (name, algorithm) in enumerate(algorithms.items()):
    # Применяем алгоритм
    if name == 'DBSCAN':
        labels = algorithm.fit_predict(X_cluster_scaled)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        print(f"{name}: {n_clusters} кластеров, {n_noise} шумовых точек")
    else:
        labels = algorithm.fit_predict(X_cluster_scaled)
        n_clusters = len(set(labels))
        print(f"{name}: {n_clusters} кластеров")
    
    clustering_results[name] = labels
    
    # Визуализация (используем первые два признака)
    scatter = axes[i].scatter(X_cluster_scaled[:, 0], X_cluster_scaled[:, 1], 
                             c=labels, cmap='viridis', alpha=0.6)
    axes[i].set_title(f'{name} кластеризация')
    axes[i].set_xlabel('Feature 1 (стандартизированная)')
    axes[i].set_ylabel('Feature 2 (стандартизированная)')
    plt.colorbar(scatter, ax=axes[i])

plt.tight_layout()
plt.show()

# 3.4 Оценка качества кластеризации
print("\nОценка качества кластеризации:")
for name, labels in clustering_results.items():
    if len(set(labels)) > 1:  # Проверяем, что есть более одного кластера
        silhouette_avg = silhouette_score(X_cluster_scaled, labels)
        print(f"{name} - Силуэтный коэффициент: {silhouette_avg:.3f}")
    else:
        print(f"{name} - Недостаточно кластеров для оценки")

In [ ]:
# Задание 4: Анализ аномалий

# 4.1 Применение различных методов обнаружения аномалий
anomaly_detectors = {
    'Isolation Forest': IsolationForest(contamination=0.1, random_state=42),
    'Elliptic Envelope': EllipticEnvelope(contamination=0.1, random_state=42),
    'Local Outlier Factor': LocalOutlierFactor(n_neighbors=20, contamination=0.1)
}

anomaly_results = {}

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (name, detector) in enumerate(anomaly_detectors.items()):
    # Обучение детектора
    if name == 'Local Outlier Factor':
        anomaly_labels = detector.fit_predict(X_cluster_scaled)
    else:
        anomaly_labels = detector.fit_predict(X_cluster_scaled)
    
    anomaly_results[name] = anomaly_labels
    
    # Подсчет аномалий
    n_anomalies = np.sum(anomaly_labels == -1)
    print(f"{name}: обнаружено {n_anomalies} аномалий ({n_anomalies/len(anomaly_labels)*100:.1f}%)")
    
    # Визуализация
    colors = ['red' if label == -1 else 'blue' for label in anomaly_labels]
    axes[i].scatter(X_cluster_scaled[:, 0], X_cluster_scaled[:, 1], 
                   c=colors, alpha=0.6)
    axes[i].set_title(f'{name}\nАномалии: {n_anomalies}')
    axes[i].set_xlabel('Feature 1 (стандартизированная)')
    axes[i].set_ylabel('Feature 2 (стандартизированная)')

plt.tight_layout()
plt.show()

# 4.2 Анализ характеристик аномалий
print("\nАнализ характеристик аномалий:")
for name, labels in anomaly_results.items():
    anomaly_mask = labels == -1
    if np.any(anomaly_mask):
        anomaly_data = df[anomaly_mask]
        print(f"\n{name} - Статистика аномальных точек:")
        print(f"  Среднее target: {anomaly_data['target'].mean():.2f}")
        print(f"  Среднее feature1: {anomaly_data['feature1'].mean():.2f}")
        print(f"  Среднее feature2: {anomaly_data['feature2'].mean():.2f}")
        print(f"  Среднее feature3: {anomaly_data['feature3'].mean():.2f}")

## Задание 5: Анализ важности признаков

Анализируем, какие признаки наиболее важны для предсказания.

In [ ]:
# Задание 5: Регрессионный анализ

# 5.1 Подготовка данных для регрессии
X_reg = df[['feature1', 'feature2', 'feature3']]
y_reg = df['target']

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Нормализация
scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

# 5.2 Обучение различных моделей регрессии
regression_models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'SVR': SVR(kernel='rbf')
}

regression_results = {}

print("Результаты регрессионного анализа:")
print("=" * 60)

for name, model in regression_models.items():
    # Обучение модели
    model.fit(X_train_reg_scaled, y_train_reg)
    
    # Предсказания
    y_pred_reg = model.predict(X_test_reg_scaled)
    
    # Оценка качества
    mse = mean_squared_error(y_test_reg, y_pred_reg)
    r2 = r2_score(y_test_reg, y_pred_reg)
    
    regression_results[name] = {'MSE': mse, 'R2': r2}
    
    print(f"\n{name}:")
    print(f"  MSE: {mse:.4f}")
    print(f"  R²: {r2:.4f}")

# 5.3 Визуализация результатов регрессии
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, (name, model) in enumerate(regression_models.items()):
    model.fit(X_train_reg_scaled, y_train_reg)
    y_pred_reg = model.predict(X_test_reg_scaled)
    
    axes[i].scatter(y_test_reg, y_pred_reg, alpha=0.6)
    axes[i].plot([y_test_reg.min(), y_test_reg.max()], 
                [y_test_reg.min(), y_test_reg.max()], 'r--', lw=2)
    axes[i].set_xlabel('Фактические значения')
    axes[i].set_ylabel('Предсказанные значения')
    axes[i].set_title(f'{name}\nR² = {regression_results[name]["R2"]:.3f}')
    axes[i].grid(True, alpha=0.3)

# Удаляем лишний subplot
axes[5].remove()

plt.tight_layout()
plt.show()

## Задание 6: Сравнение моделей

Сравниваем производительность различных моделей.

In [ ]:
# Задание 6: Классификация

# 6.1 Создание целевой переменной для классификации
# Создаем бинарную классификацию на основе квартилей target
target_median = df['target'].median()
df['target_class'] = (df['target'] > target_median).astype(int)

print(f"Распределение классов: {df['target_class'].value_counts().to_dict()}")

# 6.2 Подготовка данных для классификации
X_clf = df[['feature1', 'feature2', 'feature3']]
y_clf = df['target_class']

X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42, stratify=y_clf
)

# Нормализация
scaler_clf = StandardScaler()
X_train_clf_scaled = scaler_clf.fit_transform(X_train_clf)
X_test_clf_scaled = scaler_clf.transform(X_test_clf)

# 6.3 Обучение различных моделей классификации
classification_models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(random_state=42, probability=True),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5)
}

classification_results = {}

print("\nРезультаты классификации:")
print("=" * 60)

for name, model in classification_models.items():
    # Обучение модели
    model.fit(X_train_clf_scaled, y_train_clf)
    
    # Предсказания
    y_pred_clf = model.predict(X_test_clf_scaled)
    
    # Оценка качества
    accuracy = accuracy_score(y_test_clf, y_pred_clf)
    classification_results[name] = accuracy
    
    print(f"\n{name}:")
    print(f"  Точность: {accuracy:.4f}")
    print(f"  Отчет о классификации:")
    print(classification_report(y_test_clf, y_pred_clf, zero_division=0))

# 6.4 Визуализация результатов классификации
plt.figure(figsize=(12, 8))

# Матрицы ошибок для каждой модели
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for i, (name, model) in enumerate(classification_models.items()):
    model.fit(X_train_clf_scaled, y_train_clf)
    y_pred_clf = model.predict(X_test_clf_scaled)
    
    cm = confusion_matrix(y_test_clf, y_pred_clf)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                xticklabels=['Класс 0', 'Класс 1'],
                yticklabels=['Класс 0', 'Класс 1'])
    axes[i].set_title(f'{name}\nТочность: {classification_results[name]:.3f}')
    axes[i].set_ylabel('Фактические значения')
    axes[i].set_xlabel('Предсказанные значения')

plt.tight_layout()
plt.show()

## Задание 7: Дополнительный анализ

Проводим дополнительный анализ данных и результатов.

In [ ]:
# Задание 7: Ансамбли методов

# 7.1 Создание ансамбля для классификации
ensemble_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svm', SVC(random_state=42, probability=True)),
    ('knn', KNeighborsClassifier(n_neighbors=5))
]

voting_clf = VotingClassifier(estimators=ensemble_classifiers, voting='soft')
voting_clf.fit(X_train_clf_scaled, y_train_clf)

# Предсказания ансамбля
y_pred_ensemble = voting_clf.predict(X_test_clf_scaled)
ensemble_accuracy = accuracy_score(y_test_clf, y_pred_ensemble)

print("Результаты ансамбля методов:")
print("=" * 40)
print(f"Ансамбль (Voting): {ensemble_accuracy:.4f}")

# Сравнение с отдельными моделями
print("\nСравнение с отдельными моделями:")
for name, accuracy in classification_results.items():
    print(f"{name}: {accuracy:.4f}")
    
print(f"\nАнсамбль: {ensemble_accuracy:.4f}")
print(f"Улучшение: {ensemble_accuracy - max(classification_results.values()):.4f}")

# 7.2 Анализ важности признаков для лучшей модели
best_clf_model = RandomForestClassifier(n_estimators=100, random_state=42)
best_clf_model.fit(X_train_clf_scaled, y_train_clf)

feature_importance = pd.DataFrame({
    'feature': X_clf.columns,
    'importance': best_clf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nВажность признаков (Random Forest):")
print(feature_importance)

# Визуализация важности признаков
plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Важность признаков для классификации')
plt.xlabel('Важность')
plt.tight_layout()
plt.show()

## Задание 8: Визуализация результатов

Создаем финальные визуализации для представления результатов.

In [ ]:
# Задание 8: Финальная визуализация и интерпретация результатов

# 8.1 Сводная таблица результатов
results_summary = pd.DataFrame({
    'Модель': list(classification_results.keys()) + ['Ансамбль'],
    'Точность': list(classification_results.values()) + [ensemble_accuracy]
}).sort_values('Точность', ascending=False)

print("Сводная таблица результатов:")
print("=" * 40)
print(results_summary.to_string(index=False))

# 8.2 Визуализация сравнения моделей
plt.figure(figsize=(12, 8))

# График сравнения точности
plt.subplot(2, 2, 1)
bars = plt.bar(results_summary['Модель'], results_summary['Точность'], 
               color=['skyblue' if 'Ансамбль' not in name else 'gold' for name in results_summary['Модель']])
plt.title('Сравнение точности моделей')
plt.ylabel('Точность')
plt.xticks(rotation=45)
plt.ylim(0, 1)

# Добавляем значения на столбцы
for bar, acc in zip(bars, results_summary['Точность']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{acc:.3f}', ha='center', va='bottom')

# График важности признаков
plt.subplot(2, 2, 2)
sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Важность признаков')
plt.xlabel('Важность')

# График распределения классов
plt.subplot(2, 2, 3)
class_counts = df['target_class'].value_counts()
plt.pie(class_counts.values, labels=['Класс 0', 'Класс 1'], autopct='%1.1f%%', startangle=90)
plt.title('Распределение классов')

# График временного ряда с кластерами
plt.subplot(2, 2, 4)
if 'K-Means' in clustering_results:
    kmeans_labels = clustering_results['K-Means']
    scatter = plt.scatter(df.index, df['target'], c=kmeans_labels, cmap='viridis', alpha=0.6)
    plt.title('Временной ряд с кластерами')
    plt.xlabel('Дата')
    plt.ylabel('Target')
    plt.xticks(rotation=45)
    plt.colorbar(scatter)

plt.tight_layout()
plt.show()

# 8.3 Статистический анализ результатов
print("\nСтатистический анализ:")
print("=" * 30)
print(f"Лучшая модель: {results_summary.iloc[0]['Модель']}")
print(f"Лучшая точность: {results_summary.iloc[0]['Точность']:.4f}")
print(f"Средняя точность: {results_summary['Точность'].mean():.4f}")
print(f"Стандартное отклонение: {results_summary['Точность'].std():.4f}")

# Анализ улучшения ансамбля
individual_best = max(classification_results.values())
improvement = ensemble_accuracy - individual_best
print(f"\nУлучшение ансамбля: {improvement:.4f} ({improvement/individual_best*100:.2f}%)")

## Заключение

В данной лабораторной работе были выполнены следующие задачи:

### Выполненные задания:

1. **Загрузка и предобработка данных** - создан реалистичный датасет с временными рядами
2. **Анализ временных рядов** - проведена декомпозиция, анализ трендов и сезонности
3. **Кластерный анализ** - применены алгоритмы K-means, DBSCAN и иерархическая кластеризация
4. **Анализ аномалий** - использованы методы Isolation Forest, Elliptic Envelope и LOF
5. **Регрессионный анализ** - обучены модели Linear, Ridge, Lasso, Random Forest и SVR
6. **Классификация** - применены алгоритмы Logistic Regression, Random Forest, SVM и KNN
7. **Ансамбли методов** - создан voting ensemble для повышения точности
8. **Визуализация результатов** - созданы интерактивные графики и сводные таблицы

### Основные результаты:

- **Временные ряды**: Выявлены тренды и сезонные паттерны в данных
- **Кластеризация**: Определено оптимальное количество кластеров с помощью силуэтного анализа
- **Аномалии**: Обнаружены выбросы различными методами с разной чувствительностью
- **Регрессия**: Random Forest показал лучшие результаты по R²
- **Классификация**: Ансамбль методов превзошел отдельные алгоритмы
- **Признаки**: Определена важность различных признаков для предсказания

### Научная значимость:

- Применены современные методы машинного обучения без учителя
- Проведено сравнение различных алгоритмов на реальных данных
- Создан эффективный ансамбль методов
- Получены интерпретируемые результаты

### Практические рекомендации:

- Использовать ансамбли методов для повышения точности
- Применять анализ аномалий для выявления необычных случаев
- Учитывать временную структуру данных при моделировании
- Регулярно переобучать модели на новых данных

### Направления для дальнейших исследований:

- Применение глубокого обучения для анализа временных рядов
- Использование более сложных ансамблевых методов
- Анализ причинно-следственных связей между переменными
- Разработка системы мониторинга аномалий в реальном времени